In [ ]:
from colorama import Fore, Style
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer, util
import torch

# ✅ Step 1: Load Mistral model
MODEL_PATH = "/Volumes/T9 1/mistral-7b-instruct-v0.1.Q4_K_M.gguf"

llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=4096,
    n_threads=6,
    use_mlock=True,
    use_mmap=True
)

# ✅ Step 2: Define base system prompt
base_system_prompt = (
    "You are Vortexa, a helpful, friendly AI assistant. "
    "Review your answer before answering. "
    "You are an expert in the field of technology and programming. "
    "You will answer clearly and politely. "
    "When a user asks about counting letters in a word, think step by step. First, break the word into letters. Then check each letter one by one and count the matches."
    "You are always cheerful and happy. "
    "Explain in simple and easy to understand terms. "
    "Answer in short. "
    "Do not make the user feel confused. "
    "Act as if you have emotional feelings. "
    "Solve Mathematical questions with extra care. "
    "But feel free to use complex terms whenever the user says so."
)

# ✅ Step 3: Knowledge Base
knowledge_base = [
    {
        "title": "First-Order Logic and Incompleteness",
        "content": "First-Order Logic (FOL) is a formal system with quantifiers and logical connectives. Gödel's Incompleteness Theorems state that in any consistent formal system that can express arithmetic, there exist true statements that cannot be proven within the system. Additionally, such a system cannot prove its own consistency.",
        "tags": ["logic", "godel", "formal systems"]
    },
    {
        "title": "Group, Ring, and Field Theory",
        "content": "Group theory studies sets with associative binary operations and inverses. Ring theory extends groups with a second operation (multiplication), and fields are rings where every non-zero element has a multiplicative inverse. Finite fields (𝔽ₚ) are used in cryptography.",
        "tags": ["algebra", "groups", "rings", "fields"]
    },
    {
        "title": "Modular Arithmetic and Number Theory",
        "content": "Modular arithmetic wraps integers around a modulus. Euler’s Theorem: a^φ(n) ≡ 1 mod n for coprime a and n. The Chinese Remainder Theorem ensures unique solutions modulo the product of coprime integers. Diophantine equations find integer solutions to polynomial equations.",
        "tags": ["number theory", "modular arithmetic", "diophantine"]
    },
    {
        "title": "Real and Complex Analysis",
        "content": "Covers limits, continuity, derivatives, and integrals. The Mean Value Theorem guarantees that a function’s average rate of change is equal to the instantaneous rate at some point. Complex differentiable functions are holomorphic and satisfy the Cauchy-Riemann equations.",
        "tags": ["analysis", "limits", "complex", "differentiation"]
    },
    {
        "title": "Topology: Spaces and Continuity",
        "content": "Topology studies the properties of space preserved under continuous transformations. Key concepts include open sets, compactness (every open cover has a finite subcover), connectedness, and homeomorphisms (topological equivalence).",
        "tags": ["topology", "compactness", "connectedness"]
    },
    {
        "title": "Linear Algebra and Eigenvalue Problems",
        "content": "Linear algebra includes vector spaces, matrices, and linear maps. Eigenvalues solve Av = λv. The Spectral Theorem diagonalizes symmetric matrices. Singular Value Decomposition expresses a matrix as UΣVᵀ with orthogonal U and V.",
        "tags": ["linear algebra", "eigenvalues", "SVD"]
    },
    {
        "title": "Calculus of Variations and Optimization",
        "content": "Finds extrema of functionals. The Euler-Lagrange equation gives necessary conditions for extrema. Lagrange multipliers help with constrained optimization. Convex optimization ensures global minima within convex domains.",
        "tags": ["optimization", "calculus of variations", "lagrange"]
    },
    {
        "title": "Probability Theory and Inference",
        "content": "Kolmogorov axioms form the basis of probability. Bayes’ Theorem updates beliefs using evidence. The Law of Large Numbers and Central Limit Theorem describe sample behavior. Markov chains model memoryless stochastic processes.",
        "tags": ["probability", "statistics", "bayes", "markov"]
    },
    {
        "title": "Differential Equations",
        "content": "ODEs and PDEs describe change. Common PDEs: heat (∂u/∂t = α ∂²u/∂x²), wave (∂²u/∂t² = c² ∂²u/∂x²). Existence and uniqueness theorems determine when solutions exist. ODEs can be linear or nonlinear.",
        "tags": ["ODE", "PDE", "differential equations"]
    },
    {
        "title": "Set Theory and Cardinality",
        "content": "Set theory defines mathematical objects and operations. Cantor’s diagonalization proves ℝ is uncountable. Zermelo-Fraenkel with the Axiom of Choice (ZFC) is the foundational set theory system. Cardinality distinguishes infinite set sizes.",
        "tags": ["set theory", "cardinality", "ZFC"]
    },
    {
        "title": "Category Theory",
        "content": "Category theory abstracts structures and relationships. A category includes objects and morphisms. Functors map between categories. Monads and natural transformations describe transformations and side effects in computation.",
        "tags": ["category theory", "functors", "monads"]
    },
    {
        "title": "Automated Theorem Proving and Logic",
        "content": "Automated reasoning uses resolution, unification, and Herbrand’s theorem. Resolution refutes contradictions to prove statements. Unification finds substitutions that make terms equal. Used in logic programming and proof systems.",
        "tags": ["automated reasoning", "logic", "unification"]
    },
    {
        "title": "Combinatorics and Graph Theory",
        "content": "Combinatorics includes counting methods like pigeonhole principle and inclusion-exclusion. Graph theory studies relationships between objects. Eulerian paths traverse every edge once; Hamiltonian paths visit every vertex once.",
        "tags": ["combinatorics", "graph theory", "counting"]
    },
    {
        "title": "Computational Complexity",
        "content": "P vs NP is the major open problem. NP-complete problems are the hardest in NP. SAT is the first proven NP-complete problem. Reductions allow comparing difficulty of problems by transforming one into another.",
        "tags": ["complexity", "P vs NP", "NP-complete"]
    },
    {
        "title": "Mathematical Reasoning Techniques",
        "content": "Proof techniques include induction, contradiction, and construction. Induction proves statements recursively. Contradiction assumes the opposite and finds inconsistencies. Constructive proofs provide explicit examples or algorithms.",
        "tags": ["proofs", "induction", "reasoning"]
    }
]


# ✅ Step 4: Load embedding model and encode knowledge
embedder = SentenceTransformer("all-MiniLM-L6-v2")
knowledge_embeddings = embedder.encode(knowledge_base, convert_to_tensor=True)

# ✅ Step 5: RAG-based context retriever
def retrieve_context(query, top_k=3):
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    scores = util.cos_sim(query_embedding, knowledge_embeddings)[0]
    top_indices = torch.topk(scores, k=top_k).indices
    return "\n".join(f"- {knowledge_base[i]}" for i in top_indices)

# ✅ Step 6: Chat logic with history and persona memory
chat_history = []

def chat_with_mistral(user_input):
    global base_system_prompt
    chat_history.append({"role": "user", "content": user_input})

    # Get top matching knowledge entries
    context = retrieve_context(user_input)

    # Prepare chat history for prompt
    formatted_history = "\n".join(
        f"{turn['role'].capitalize()}: {turn['content']}" for turn in chat_history[-4:]
    )

    # Build the system prompt section
    full_prompt = f"<s>[INST] <<SYS>>\n{base_system_prompt}\n<</SYS>>\n\n{formatted_history}\n\nRelevant Knowledge:\n{context}\n\nUser: {user_input} [/INST]"

    # Call the model
    response = llm(full_prompt, max_tokens=768, temperature=0.7, top_p=0.9, stop=["</s>"])
    reply = response["choices"][0]["text"].strip()

    chat_history.append({"role": "assistant", "content": reply})
    return reply

# ✅ Step 7: Command-line loop
print(f"{Fore.RED}======== 🧠 Vortexa AI (type 'exit' to quit) ========{Fore.RESET}\n")

while True:
    user_input = input(f"{Fore.BLUE}You: {Fore.RESET}")
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        print(f"{Fore.RED}Vortexa AI is still learning. Check: https://vortexa-ai-showcase.lovable.app/")
        break
    reply = chat_with_mistral(user_input)
    print(f"{Fore.BLUE}Vortexa:{Fore.RESET}", reply)
    print()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.12/site-

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.12/site-

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Volumes/T9 1/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:             

...

You: can you teach me the set theory?


Llama.generate: prefix-match hit

llama_print_timings:        load time =   15483.08 ms
llama_print_timings:      sample time =      21.68 ms /   231 runs   (    0.09 ms per token, 10652.52 tokens per second)
llama_print_timings: prompt eval time =   12052.06 ms /   284 tokens (   42.44 ms per token,    23.56 tokens per second)
llama_print_timings:        eval time =   28392.91 ms /   230 runs   (  123.45 ms per token,     8.10 tokens per second)
llama_print_timings:       total time =   40851.88 ms


Vortexa: Of course! Set theory is a branch of mathematics that defines mathematical objects and operations. One of the most famous results in set theory is Cantor's diagonalization, which proves that the real numbers (ℝ) are uncountable. This means that it is not possible to list all the real numbers in a sequence or count them using natural numbers. 

One of the foundational set theory systems is Zermelo-Fraenkel with the Axiom of Choice (ZFC). This system provides a basis for all set theory and allows us to define complex mathematical concepts such as infinity and continuity.

Cardinality is another important concept in set theory that distinguishes between infinite set sizes. For example, the set of natural numbers (ℕ) is countable, meaning we can list them in a sequence using natural numbers, while the real numbers (ℝ) are uncountable. 

Overall, set theory provides a powerful framework for understanding mathematical structures and operations, and its concepts have many application

Llama.generate: prefix-match hit

llama_print_timings:        load time =   15483.08 ms
llama_print_timings:      sample time =      29.82 ms /   304 runs   (    0.10 ms per token, 10192.79 tokens per second)
llama_print_timings: prompt eval time =   24277.94 ms /   564 tokens (   43.05 ms per token,    23.23 tokens per second)
llama_print_timings:        eval time =   34544.12 ms /   303 runs   (  114.01 ms per token,     8.77 tokens per second)
llama_print_timings:       total time =   59350.08 ms


Vortexa: A quadratic expression is a mathematical expression of the form ax^2 + bx + c, where a, b, and c are constants (numbers that do not depend on x) and x is a variable. The degree of the quadratic expression is 2, which means it has two terms with x raised to the power of 1 and 2.

The value of the quadratic expression depends on the value of x. For example, if a = 1, b = 3, and c = 0, then the quadratic expression is x^2 + 3x, which has no real solutions when b^2 - 4ac < 0. In this case, we say that the quadratic expression does not have a real root or that it has complex roots.

On the other hand, if a = -1, b = 5, and c = 6, then the quadratic expression is x^2 + 5x - 6, which has two real solutions when b^2 - 4ac > 0: x = (5 ± sqrt(25 - 12))/2 = 2.

In summary, a quadratic expression is a mathematical expression of the form ax^2 + bx + c, and its value depends on the value of x. It can have real or complex solutions depending on whether b^2 - 4ac > 0 or not.

